# **Flight Delay Prediction using Scikit-Learn Pipeline**

## **Overview**
This project demonstrates how to build a **machine learning pipeline** using scikit-learn to predict flight delays. The pipeline integrates data preprocessing with model training, ensuring efficient handling of both numerical and categorical data.

---

## **Objectives**
- Preprocess numerical and categorical data using `ColumnTransformer`.
- Automate the machine learning workflow using `Pipeline`.
- Train a **Random Forest Classifier** to predict flight delays.
- Optimize the model using **GridSearchCV** for hyperparameter tuning.

---

## **Data Overview**
- **Dataset**: Contains flight details such as:
  - **Year**, **Month**, **Day**
  - **Airline code**, **Origin airport code**, **Destination airport code**
  - **Departure delay** (target: delayed or not)

- **Target Variable**:  
  - `1` if the flight was delayed  
  - `0` if the flight was on time

---

## **Steps Involved**

### 1. **Data Loading and Exploration**
- Load the flight dataset and inspect its structure and missing values.

### 2. **Feature Engineering**
- **Numerical Features**:
  - `YEAR`, `MONTH`, `DAY`
- **Categorical Features**:
  - `AIRLINE__CODE`, `ORIGIN_AIRPORT_CODE`, `DESTINATION_AIRPORT_CODE`

### 3. **Preprocessing with `ColumnTransformer`**
- **Numerical Data**:
  - Impute missing values with the **mean**.
  - Standardize values using **`StandardScaler`**.
  
- **Categorical Data**:
  - Impute missing values with `'missing'`.
  - Encode using **`OneHotEncoder`**.

### 4. **Pipeline Setup**
- Use a **scikit-learn Pipeline** to link preprocessing and model training.
- Integrate a **Random Forest Classifier** within the pipeline.

### 5. **Model Training and Evaluation**
- Split the data into **train (70%)** and **test (30%)** sets.
- Evaluate the model using a **classification report** with metrics like:
  - **Precision**, **Recall**, **F1-score**

### 6. **Hyperparameter Tuning with GridSearchCV**
- Tune hyperparameters of the Random Forest model:
  - Number of estimators (`n_estimators`)
  - Maximum tree depth (`max_depth`)

### 7. **Model Persistence**
- Save the trained model using **`joblib`** for later use.

---

## **Technologies Used**
- **Python**: Programming language
- **Pandas**: Data manipulation and cleaning
- **Scikit-Learn**: Machine learning, preprocessing, and model evaluation
- **Joblib**: Model persistence
- **Jupyter Notebook**: Interactive development environment

---

## **Expected Output**
- A **trained Random Forest model** to predict flight delays.
- **Performance metrics** (accuracy, precision, recall) from the classification report.
- A **saved model** (`flight_delay_classifier.pkl`) for deployment.

---

## **Conclusion**
This project demonstrates how to create an automated **machine learning workflow** using scikit-learn’s `Pipeline` and `ColumnTransformer`. The streamlined preprocessing ensures consistency during both training and testing. With **hyperparameter tuning**, the model's performance is further optimized, making it reliable for real-world flight delay predictions.


# Upgrade pip and install all required packages

In [ ]:
!pip install --upgrade pip

# Install Snowflake connectors, pandas integration, and essential libraries
!pip install "snowflake-connector-python[pandas]" \
             snowflake-snowpark-python==1.9.0 \
             numpy pandas matplotlib scikit-learn xgboost seaborn \
             python-dateutil tqdm holidays faker

# Ensure Snowpark Python is up-to-date
!pip install --upgrade -q snowflake-snowpark-python==1.9.0

# Fix potential urllib3 version conflicts
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

# Additional installations for your project
!pip install fosforml==1.1.6
!pip install python-scipy
!pip install basemap


# Importing necessary libraries and settings

In [2]:

# Standard libraries for date and warnings
import datetime
import warnings

# Scientific and Data Manipulation Libraries
import scipy
import pandas as pd
import numpy as np

# Data Visualization Libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

# Sklearn Modules for Data Preprocessing, Modeling, and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder  # Encoding categorical variables
from sklearn.preprocessing import StandardScaler  # Scaling numerical data
from sklearn.tree import DecisionTreeClassifier  # Decision Tree model
from sklearn.metrics import roc_auc_score, classification_report  # Evaluation metrics

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

import configparser
from dateutil.relativedelta import relativedelta
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

# Configuring display options and warning filters
pd.options.display.max_columns = 50
warnings.filterwarnings("ignore")

# Custom FosforML package for Snowflake session and model registration
from fosforml.model_manager.snowflakesession import get_session
from fosforml import register_model
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer


In [3]:
# Set Matplotlib's default font family to 'DeJavu Serif' to ensure a consistent font style across plots
plt.rcParams['font.family'] = 'DeJavu Serif'

# Establishing a Snowflake session


In [4]:
my_session = get_session()

# Defining the table name to fetch data from
# table_name = 'FLIGHTS'  # Initial option for table
table_name = 'FLIGHTS_FULL'  # Final table to be used

# Querying the data from the specified Snowflake table
sf_df = my_session.sql("SELECT * FROM {}".format(table_name))

# Converting the Snowflake DataFrame to a pandas DataFrame for local processing
df = sf_df.to_pandas()

df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLY_DATE,AIRLINE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE
0,2024,4,23,4,AA,1114,N3HTAA,LGA,DFW,1529,1542.0,13.0,18.0,1600.0,242.0,232.0,209.0,1389,1829.0,5.0,1831,1834.0,3.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,American Airlines Inc.,LaGuardia Airport (Marine Air Terminal),New York,NY,USA,40.77724,-73.87261,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720
1,2024,4,23,4,B6,813,N775JB,HPN,FLL,1529,1520.0,-9.0,15.0,1535.0,184.0,178.0,152.0,1097,1807.0,11.0,1833,1818.0,-15.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,JetBlue Airways,Westchester County Airport,White Plains,NY,USA,41.06696,-73.70757,Fort Lauderdale-Hollywood International Airport,Ft. Lauderdale,FL,USA,26.07258,-80.15275
2,2024,4,23,4,DL,1612,N942DL,ORD,ATL,1529,1529.0,0.0,11.0,1540.0,120.0,95.0,78.0,606,1758.0,6.0,1829,1804.0,-25.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Delta Air Lines Inc.,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
3,2024,4,23,4,DL,1642,N906AT,ATL,CAK,1529,1526.0,-3.0,17.0,1543.0,104.0,102.0,80.0,528,1703.0,5.0,1713,1708.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Delta Air Lines Inc.,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Akron-Canton Regional Airport,Akron,OH,USA,40.91631,-81.44247
4,2024,4,23,4,EV,4973,N931EV,ATL,GTR,1530,1528.0,-2.0,27.0,1555.0,69.0,75.0,45.0,241,1540.0,3.0,1539,1543.0,4.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Atlantic Southeast Airlines,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Golden Triangle Regional Airport,Columbus-Starkville-West Point,MS,USA,33.45033,-88.59137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2024,7,9,4,OO,6477,N117SY,ORD,DFW,1551,1548.0,-3.0,19.0,1607.0,149.0,175.0,145.0,802,1832.0,11.0,1820,1843.0,23.0,0,0,None,23.0,0.0,0.0,0.0,0.0,2024-07-09,Skywest Airlines Inc.,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720
5819075,2024,7,9,4,OO,6498,N110SY,BOI,LAX,1551,1610.0,19.0,12.0,1622.0,139.0,126.0,103.0,674,1705.0,11.0,1710,1716.0,6.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-07-09,Skywest Airlines Inc.,Boise Airport (Boise Air Terminal),Boise,ID,USA,43.56444,-116.22278,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807
5819076,2024,7,9,4,EV,2675,N908EV,JAN,DFW,1551,1550.0,-1.0,10.0,1600.0,94.0,94.0,70.0,408,1710.0,14.0,1725,1724.0,-1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-07-09,Atlantic Southeast Airlines,Jackson-Evers International Airport,Jackson,MS,USA,32.31117,-90.07589,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720
5819077,2024,7,9,4,EV,5135,N759EV,RST,MSP,1551,1548.0,-3.0,24.0,1612.0,46.0,56.0,25.0,76,1637.0,7.0,1637,1644.0,7.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-07-09,Atlantic Southeast Airlines,Rochester International Airport,Rochester,MN,USA,43.90883,-92.49799,Minneapolis-Saint Paul International Airport,Minneapolis,MN,USA,44.88055,-93.21692


# Filtering data for specific airlines

In [5]:
# Defining the list of airlines to include in the filtered DataFrame
options = ['Southwest Airlines Co.', 'Delta Air Lines Inc.']

# Selecting rows where the 'AIRLINE' column matches one of the specified airlines
flights = df.loc[df['AIRLINE'].isin(options)]
flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLY_DATE,AIRLINE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE
2,2024,4,23,4,DL,1612,N942DL,ORD,ATL,1529,1529.0,0.0,11.0,1540.0,120.0,95.0,78.0,606,1758.0,6.0,1829,1804.0,-25.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Delta Air Lines Inc.,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
3,2024,4,23,4,DL,1642,N906AT,ATL,CAK,1529,1526.0,-3.0,17.0,1543.0,104.0,102.0,80.0,528,1703.0,5.0,1713,1708.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Delta Air Lines Inc.,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Akron-Canton Regional Airport,Akron,OH,USA,40.91631,-81.44247
39,2024,4,23,4,WN,395,N7740A,LAS,SAN,1530,1555.0,25.0,26.0,1621.0,65.0,74.0,45.0,258,1706.0,3.0,1635,1709.0,34.0,0,0,None,9.0,0.0,5.0,20.0,0.0,2024-04-23,Southwest Airlines Co.,McCarran International Airport,Las Vegas,NV,USA,36.08036,-115.15233,San Diego International Airport (Lindbergh Field),San Diego,CA,USA,32.73356,-117.18966
40,2024,4,23,4,WN,4989,N647SW,ATL,IND,1530,1531.0,1.0,18.0,1549.0,100.0,96.0,73.0,432,1702.0,5.0,1710,1707.0,-3.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Southwest Airlines Co.,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Indianapolis International Airport,Indianapolis,IN,USA,39.71733,-86.29438
41,2024,4,23,4,WN,1878,N8642E,SEA,DEN,1530,1537.0,7.0,13.0,1550.0,165.0,147.0,128.0,1024,1858.0,6.0,1915,1904.0,-11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Southwest Airlines Co.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,Denver International Airport,Denver,CO,USA,39.85841,-104.66700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819055,2024,7,9,4,WN,268,N266WN,SNA,OAK,1550,1546.0,-4.0,10.0,1556.0,80.0,81.0,65.0,371,1701.0,6.0,1710,1707.0,-3.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-07-09,Southwest Airlines Co.,John Wayne Airport (Orange County Airport),Santa Ana,CA,USA,33.67566,-117.86822,Oakland International Airport,Oakland,CA,USA,37.72129,-122.22072
5819056,2024,7,9,4,WN,2427,N270WN,STL,DCA,1550,1608.0,18.0,11.0,1619.0,115.0,137.0,121.0,719,1920.0,5.0,1845,1925.0,40.0,0,0,None,22.0,0.0,0.0,18.0,0.0,2024-07-09,Southwest Airlines Co.,St. Louis International Airport at Lambert Field,St Louis,MO,USA,38.74769,-90.35999,Ronald Reagan Washington National Airport,Arlington,VA,USA,38.85208,-77.03772
5819057,2024,7,9,4,WN,2602,N796SW,ORF,BWI,1550,1602.0,12.0,13.0,1615.0,55.0,52.0,35.0,159,1650.0,4.0,1645,1654.0,9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-07-09,Southwest Airlines Co.,Norfolk International Airport,Norfolk,VA,USA,36.89461,-76.20122,Baltimore-Washington International Airport,Baltimore,MD,USA,39.17540,-76.66820
5819071,2024,7,9,4,DL,1543,N341NB,DTW,OMA,1551,1548.0,-3.0,15.0,1603.0,121.0,111.0,94.0,651,1637.0,2.0,1652,1639.0,-13.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-07-09,Delta Air Lines Inc.,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,Eppley Airfield,Omaha,NE,USA,41.30252,-95.89417


# Creating a copy of the filtered flights data

In [6]:
# This ensures that any modifications made to 'flights_needed_data' do not affect the original 'flights' DataFrame
flights_needed_data = flights.copy()

In [7]:
flights_needed_data.shape

(2137736, 45)

In [8]:
flights_needed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2137736 entries, 2 to 5819072
Data columns (total 45 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   YEAR                      int16  
 1   MONTH                     int8   
 2   DAY                       int8   
 3   DAY_OF_WEEK               int8   
 4   AIRLINE__CODE             object 
 5   FLIGHT_NUMBER             int16  
 6   TAIL_NUMBER               object 
 7   ORIGIN_AIRPORT_CODE       object 
 8   DESTINATION_AIRPORT_CODE  object 
 9   SCHEDULED_DEPARTURE       int16  
 10  DEPARTURE_TIME            float64
 11  DEPARTURE_DELAY           float64
 12  TAXI_OUT                  float64
 13  WHEELS_OFF                float64
 14  SCHEDULED_TIME            float64
 15  ELAPSED_TIME              float64
 16  AIR_TIME                  float64
 17  DISTANCE                  int16  
 18  WHEELS_ON                 float64
 19  TAXI_IN                   float64
 20  SCHEDULED_ARRIVAL         int

In [9]:
flights_needed_data.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLY_DATE,AIRLINE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE
2,2024,4,23,4,DL,1612,N942DL,ORD,ATL,1529,1529.0,0.0,11.0,1540.0,120.0,95.0,78.0,606,1758.0,6.0,1829,1804.0,-25.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Delta Air Lines Inc.,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694
3,2024,4,23,4,DL,1642,N906AT,ATL,CAK,1529,1526.0,-3.0,17.0,1543.0,104.0,102.0,80.0,528,1703.0,5.0,1713,1708.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Delta Air Lines Inc.,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Akron-Canton Regional Airport,Akron,OH,USA,40.91631,-81.44247
39,2024,4,23,4,WN,395,N7740A,LAS,SAN,1530,1555.0,25.0,26.0,1621.0,65.0,74.0,45.0,258,1706.0,3.0,1635,1709.0,34.0,0,0,None,9.0,0.0,5.0,20.0,0.0,2024-04-23,Southwest Airlines Co.,McCarran International Airport,Las Vegas,NV,USA,36.08036,-115.15233,San Diego International Airport (Lindbergh Field),San Diego,CA,USA,32.73356,-117.18966
40,2024,4,23,4,WN,4989,N647SW,ATL,IND,1530,1531.0,1.0,18.0,1549.0,100.0,96.0,73.0,432,1702.0,5.0,1710,1707.0,-3.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Southwest Airlines Co.,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Indianapolis International Airport,Indianapolis,IN,USA,39.71733,-86.29438
41,2024,4,23,4,WN,1878,N8642E,SEA,DEN,1530,1537.0,7.0,13.0,1550.0,165.0,147.0,128.0,1024,1858.0,6.0,1915,1904.0,-11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Southwest Airlines Co.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,Denver International Airport,Denver,CO,USA,39.85841,-104.66700


# Function to categorize scheduled arrival times into time segments

In [10]:
def categorize_time(SCHEDULED_ARRIVAL):
    # Categorize based on scheduled arrival time in 24-hour format
    if 500 <= SCHEDULED_ARRIVAL < 800:
        return 'Early morning'
    elif 800 <= SCHEDULED_ARRIVAL < 1100:
        return 'Late morning'
    elif 1100 <= SCHEDULED_ARRIVAL < 1400:
        return 'Around noon'
    elif 1400 <= SCHEDULED_ARRIVAL < 1700:
        return 'Afternoon'
    elif 1700 <= SCHEDULED_ARRIVAL < 2000:
        return 'Evening'
    elif 2000 <= SCHEDULED_ARRIVAL < 2300:
        return 'Night'
    elif SCHEDULED_ARRIVAL >= 2300 or SCHEDULED_ARRIVAL < 200:
        return 'Late night'
    elif 200 <= SCHEDULED_ARRIVAL < 500:
        return 'Dawn'

# Apply categorize_time function to the 'SCHEDULED_ARRIVAL' column to create 'ARRIVAL_TIME_SEGMENT'
flights_needed_data['ARRIVAL_TIME_SEGMENT'] = flights_needed_data['SCHEDULED_ARRIVAL'].apply(categorize_time)


In [11]:
flights_needed_data

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLY_DATE,AIRLINE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE,ARRIVAL_TIME_SEGMENT
2,2024,4,23,4,DL,1612,N942DL,ORD,ATL,1529,1529.0,0.0,11.0,1540.0,120.0,95.0,78.0,606,1758.0,6.0,1829,1804.0,-25.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Delta Air Lines Inc.,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Evening
3,2024,4,23,4,DL,1642,N906AT,ATL,CAK,1529,1526.0,-3.0,17.0,1543.0,104.0,102.0,80.0,528,1703.0,5.0,1713,1708.0,-5.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Delta Air Lines Inc.,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Akron-Canton Regional Airport,Akron,OH,USA,40.91631,-81.44247,Evening
39,2024,4,23,4,WN,395,N7740A,LAS,SAN,1530,1555.0,25.0,26.0,1621.0,65.0,74.0,45.0,258,1706.0,3.0,1635,1709.0,34.0,0,0,None,9.0,0.0,5.0,20.0,0.0,2024-04-23,Southwest Airlines Co.,McCarran International Airport,Las Vegas,NV,USA,36.08036,-115.15233,San Diego International Airport (Lindbergh Field),San Diego,CA,USA,32.73356,-117.18966,Afternoon
40,2024,4,23,4,WN,4989,N647SW,ATL,IND,1530,1531.0,1.0,18.0,1549.0,100.0,96.0,73.0,432,1702.0,5.0,1710,1707.0,-3.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Southwest Airlines Co.,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Indianapolis International Airport,Indianapolis,IN,USA,39.71733,-86.29438,Evening
41,2024,4,23,4,WN,1878,N8642E,SEA,DEN,1530,1537.0,7.0,13.0,1550.0,165.0,147.0,128.0,1024,1858.0,6.0,1915,1904.0,-11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,Southwest Airlines Co.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,Denver International Airport,Denver,CO,USA,39.85841,-104.66700,Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819055,2024,7,9,4,WN,268,N266WN,SNA,OAK,1550,1546.0,-4.0,10.0,1556.0,80.0,81.0,65.0,371,1701.0,6.0,1710,1707.0,-3.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-07-09,Southwest Airlines Co.,John Wayne Airport (Orange County Airport),Santa Ana,CA,USA,33.67566,-117.86822,Oakland International Airport,Oakland,CA,USA,37.72129,-122.22072,Evening
5819056,2024,7,9,4,WN,2427,N270WN,STL,DCA,1550,1608.0,18.0,11.0,1619.0,115.0,137.0,121.0,719,1920.0,5.0,1845,1925.0,40.0,0,0,None,22.0,0.0,0.0,18.0,0.0,2024-07-09,Southwest Airlines Co.,St. Louis International Airport at Lambert Field,St Louis,MO,USA,38.74769,-90.35999,Ronald Reagan Washington National Airport,Arlington,VA,USA,38.85208,-77.03772,Evening
5819057,2024,7,9,4,WN,2602,N796SW,ORF,BWI,1550,1602.0,12.0,13.0,1615.0,55.0,52.0,35.0,159,1650.0,4.0,1645,1654.0,9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-07-09,Southwest Airlines Co.,Norfolk International Airport,Norfolk,VA,USA,36.89461,-76.20122,Baltimore-Washington International Airport,Baltimore,MD,USA,39.17540,-76.66820,Afternoon
5819071,2024,7,9,4,DL,1543,N341NB,DTW,OMA,1551,1548.0,-3.0,15.0,1603.0,121.0,111.0,94.0,651,1637.0,2.0,1652,1639.0,-13.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-07-09,Delta Air Lines Inc.,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,Eppley Airfield,Omaha,NE,USA,41.30252,-95.89417,Afternoon


In [12]:
flights['AIRLINE__CODE'].unique()

array(['DL', 'WN'], dtype=object)

In [13]:
flights_needed_data.value_counts('DIVERTED')

DIVERTED
0    2132545
1       5191
Name: count, dtype: int64

In [15]:
flights_needed_data['FLIGHT_NUMBER'] = flights_needed_data['FLIGHT_NUMBER'].astype(str)

In [17]:
flights_needed_data['MONTH'] = flights_needed_data['MONTH'].astype(str)
flights_needed_data['DAY'] = flights_needed_data['DAY'].astype(str)
flights_needed_data['DAY_OF_WEEK'] = flights_needed_data['DAY_OF_WEEK'].astype(str)
flights_needed_data['DIVERTED'] = flights_needed_data['DIVERTED'].astype(str)
flights_needed_data['CANCELLED'] = flights_needed_data['CANCELLED'].astype(str)
flights_needed_data['CANCELLED'] = flights_needed_data['CANCELLED'].astype(str)

In [18]:
flights_needed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2137736 entries, 2 to 5819072
Data columns (total 46 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   YEAR                      int16  
 1   MONTH                     object 
 2   DAY                       object 
 3   DAY_OF_WEEK               object 
 4   AIRLINE__CODE             object 
 5   FLIGHT_NUMBER             object 
 6   TAIL_NUMBER               object 
 7   ORIGIN_AIRPORT_CODE       object 
 8   DESTINATION_AIRPORT_CODE  object 
 9   SCHEDULED_DEPARTURE       int16  
 10  DEPARTURE_TIME            float64
 11  DEPARTURE_DELAY           float64
 12  TAXI_OUT                  float64
 13  WHEELS_OFF                float64
 14  SCHEDULED_TIME            float64
 15  ELAPSED_TIME              float64
 16  AIR_TIME                  float64
 17  DISTANCE                  int16  
 18  WHEELS_ON                 float64
 19  TAXI_IN                   float64
 20  SCHEDULED_ARRIVAL         int

In [19]:
flights_needed_data.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE__CODE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT_CODE', 'DESTINATION_AIRPORT_CODE',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'FLY_DATE', 'AIRLINE',
       'ORIGIN_AIRPORT', 'ORIGIN_CITY', 'ORIGIN_STATE', 'ORIGIN_COUNTRY',
       'ORIGIN_LATITUDE', 'ORIGIN_LONGITUDE', 'DEST_AIRPORT', 'DEST_CITY',
       'DEST_STATE', 'DEST_COUNTRY', 'DEST_LATITUDE', 'DEST_LONGITUDE',
       'ARRIVAL_TIME_SEGMENT'],
      dtype='object')

# Define columns by data type

In [21]:
##numerical_cols = ['MONTH', 'DAY', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY',
 #                 'DISTANCE', 'SCHEDULED_ARRIVAL', 'DIVERTED', 'CANCELLED', 'AIR_SYSTEM_DELAY',
 #                 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
##categorical_cols = ['AIRLINE', 'ARRIVAL_TIME_SEGMENT']

numerical_cols = ['SCHEDULED_DEPARTURE', 'DISTANCE','SCHEDULED_ARRIVAL','ARRIVAL_DELAY']                
categorical_cols = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'FLIGHT_NUMBER','TAIL_NUMBER','DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
                    'AIRLINE', 'ORIGIN_AIRPORT', 'ORIGIN_CITY', 'DEST_AIRPORT', 'DEST_CITY','ARRIVAL_TIME_SEGMENT']


# Define transformations for numerical columns: imputing and scaling

In [22]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define transformations for categorical columns: imputing and one-hot encoding


In [23]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine both transformations in a ColumnTransformer


In [24]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Create a full pipeline; add your model at the end (e.g., DecisionTreeClassifier)

In [25]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SCHEDULED_DEPARTURE',
                                                   'DISTANCE',
                                                   'SCHEDULED_ARRIVAL',
                                                   'ARRIVAL_DELAY']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['MONTH', 'DAY',
                                                   'DAY_OF_WEEK',
                                                   'FLIGHT_NUMBER',
                                                   'TAIL_NUMBER', 'DIVERTED',
                                                   'CANCELLED',
                                                   'CANCELLATION_REASON',
                                                   'AIRLINE', 'ORIGIN_AIRPORT',
                                                   'ORIGIN_CITY',
                                                   'DEST_AIRPORT', 'DEST_CITY',
                                                   'ARRIVAL_TIME_SEGMENT'])])),
                ('regressor', RandomForestRegressor())])

# Creating the target column

In [ ]:
result = []
for row in flights_needed_data['ARRIVAL_DELAY']:
  if row > 5:
    result.append(1)
  else:
    result.append(0) 

flights_needed_data['delay_flag'] = result
flights_needed_data.value_counts('delay_flag')

In [27]:
#test_data = flights_needed_data[flights_needed_data['MONTH'] >= 11] 
#test=test.drop(['FLY_DATE'], axis=1)
#train_data = flights_needed_data[flights_needed_data['MONTH'] < 11]


#test_data = flights_needed_data[flights_needed_data['MONTH'] >= 11] 
#test=test.drop(['FLY_DATE'], axis=1)
#train_data = flights_needed_data[flights_needed_data['MONTH'] < 11]

train_data  = flights_needed_data[flights_needed_data["MONTH"].apply(lambda x: '11' not in x and '12' not in x)]

test_data = flights_needed_data[flights_needed_data["MONTH"].apply(lambda x: '11' in x or '12' in x)]

In [28]:
# Replace 'target_column_name' with the actual name of your target column
X_train = train_data.drop(columns=['ARRIVAL_DELAY'])
y_train = train_data['ARRIVAL_DELAY']

In [29]:
# Replace 'target_column_name' with the actual name of your target column
X_test = test_data.drop(columns=['ARRIVAL_DELAY'])
y_test = test_data['ARRIVAL_DELAY']

In [30]:
pipeline.fit(X_train,y_train)

ValueError: A given column is not a column of the dataframe

In [ ]:
y_prob = pipeline.predict_proba(X_test)[:,1]

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
auc_score = roc_auc_score(y_test, y_pred)
auc_score

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
y_pred = pipeline.predict(flights_needed_data)

In [ ]:
flights['ACTUAL_DELAY'] = flights_needed_data['result']
flights

In [ ]:
flights['PREDICTED_DELAY'] = y_pred

In [ ]:
flights

In [ ]:
    # Ensure no NaN values remain by explicitly replacing NaN with None
my_session.write_pandas(
        df=flights,
        table_name="DELAY_CLASSIFIER_OUTPUT_2510",
        database="TTH_DB",
        schema="TTH_AIRLINE_SCHEMA",
        auto_create_table=True
)

In [ ]:
flights = flights.where(pd.notnull(flights), None)

In [ ]:
flights.isna().sum()

In [ ]:
chunk_size = 1000000
chunks = [flights[i:i + chunk_size] for i in range(0, len(flights), chunk_size)]

for chunk in chunks:
    # Ensure no NaN values remain by explicitly replacing NaN with None
    chunk = chunk.where(pd.notnull(chunk), None)
    
    # # Explicitly set the data type of each column to object
    # for col in chunk.columns:
    #     chunk[col] = chunk[col].astype(object)

    # Convert the DataFrame to a Snowflake-compatible DataFrame
    ins_train_sf = my_session.createDataFrame(
        chunk.values.tolist(),
        schema=chunk.columns.tolist()
    )
    
    # Write to Snowflake
    ins_train_sf.write.mode("append").save_as_table("TTH_DB.TTH_AIRLINE_SCHEMA.DELAY_CLASSIFIER_OUTPUT_2510")

In [ ]:
len(chunks)

In [ ]:
flights.info()

In [ ]:
my_session.sql('''
CREATE TABLE IF NOT EXISTS TTH_DB.TTH_AIRLINE_SCHEMA.DELAY_CLASSIFIER_OUTPUT_2510 (
    YEAR                      INT  NULL,
    MONTH                     INT NULL,
    DAY                       INT NULL,
    DAY_OF_WEEK               INT NULL,
    AIRLINE__CODE             STRING NULL,
    FLIGHT_NUMBER             INT NULL,
    TAIL_NUMBER               STRING NULL,
    ORIGIN_AIRPORT_CODE       STRING NULL,
    DESTINATION_AIRPORT_CODE  STRING NULL,
    SCHEDULED_DEPARTURE       INT NULL,
    DEPARTURE_TIME            FLOAT NULL,
    DEPARTURE_DELAY           FLOAT NULL,
    TAXI_OUT                  FLOAT NULL,
    WHEELS_OFF                FLOAT NULL,
    SCHEDULED_TIME            FLOAT NULL,
    ELAPSED_TIME              FLOAT NULL,
    AIR_TIME                  FLOAT NULL,
    DISTANCE                  INT NULL,
    WHEELS_ON                 FLOAT NULL,
    TAXI_IN                   FLOAT NULL,
    SCHEDULED_ARRIVAL         INT NULL,
    ARRIVAL_TIME              FLOAT NULL,
    ARRIVAL_DELAY             FLOAT NULL,
    DIVERTED                  INT NULL,
    CANCELLED                 INT NULL,
    CANCELLATION_REASON       STRING NULL,
    AIR_SYSTEM_DELAY          FLOAT NULL,
    SECURITY_DELAY            FLOAT NULL,
    AIRLINE_DELAY             FLOAT NULL,
    LATE_AIRCRAFT_DELAY       FLOAT NULL,
    WEATHER_DELAY             FLOAT NULL,
    FLY_DATE                  STRING NULL,
    AIRLINE                   STRING NULL,
    ORIGIN_AIRPORT            STRING NULL,
    ORIGIN_CITY               STRING NULL,
    ORIGIN_STATE              STRING NULL,
    ORIGIN_COUNTRY            STRING NULL,
    ORIGIN_LATITUDE           FLOAT NULL,
    ORIGIN_LONGITUDE          FLOAT NULL,
    DEST_AIRPORT              STRING NULL,
    DEST_CITY                 STRING NULL,
    DEST_STATE                STRING NULL,
    DEST_COUNTRY              STRING NULL,
    DEST_LATITUDE             FLOAT NULL,
    DEST_LONGITUDE            FLOAT NULL,
    ACTUAL_DELAY              INT NULL,
    PREDICTED_DELAY           INT NULL
);

''').collect()

In [ ]:
my_session.sql('drop table  TTH_DB.TTH_AIRLINE_SCHEMA.DELAY_CLASSIFIER_OUTPUT_2510;').collect()

In [ ]:
my_session = get_session()

# Defining the table name to fetch data from
# table_name = 'FLIGHTS'  # Initial option for table
table_name = 'DELAY_CLASSIFIER_OUTPUT_2510'  # Final table to be used

# Querying the data from the specified Snowflake table
sf_df = my_session.sql("SELECT * FROM {}".format(table_name))

# Converting the Snowflake DataFrame to a pandas DataFrame for local processing
df = sf_df.to_pandas()

df

In [ ]:
df.isna().sum()